# Training

In [ ]:
import pandas as pd
import numpy as np
import os
import random

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

import importlib
import train_models_util
import feature_importance
importlib.reload(train_models_util)
importlib.reload(feature_importance)
from train_models_util import (
    train_xgb_optuna, 
    train_catboost_optuna, 
    train_lgbm_optuna,
    train_best_base_models_from_mlflow,
    train_ensemble, 
    train_neural_network_ensemble
)
from evaluate_models_util import evaluate_and_log
from feature_importance import select_important_features

# set all random seeds for reproducibility
np.random.seed(42)
random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)

In [36]:
EXPERIMENT_NAME = "FRAUD_DETECTION_WITH_CUSTOM_THRESHOLD"

In [4]:
data_folder = 'ieee-fraud-detection-data/processed/'

train = pd.read_csv(f'{data_folder}train_processed.csv')
target = pd.read_csv(f'{data_folder}test_processed.csv')
test = pd.read_csv(f'{data_folder}target.csv')

## 1. Full Features

### 1.1. XGBoost

In [37]:
model_xgboost_full, best_params_xgboost_full, X_va_xgboost_full, y_va_xgboost_full, hist_df, plot_paths = train_xgb_optuna(train, target, early_stopping_rounds=10)

[I 2025-11-19 12:58:36,844] A new study created in memory with name: xgboost_customloss_optimization


[Optuna XGBoost Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-11-19 12:59:37,339] Trial 0 finished with value: 0.9650108375375834 and parameters: {'weight_factor': 243.75383200460732, 'learning_rate': 0.012220672042350205, 'max_depth': 12, 'subsample': 0.6101865703997771, 'colsample_bytree': 0.7393885389562767, 'min_child_weight': 0.006705852262038931, 'gamma': 2.8877876333138737}. Best is trial 0 with value: 0.9650108375375834.
[I 2025-11-19 13:00:34,531] Trial 1 finished with value: 0.9650108375375834 and parameters: {'weight_factor': 583.2965283212329, 'learning_rate': 0.013622508320752293, 'max_depth': 8, 'subsample': 0.8369564357037517, 'colsample_bytree': 0.5898788653211003, 'min_child_weight': 0.3620949567476594, 'gamma': 3.4034588716685326}. Best is trial 0 with value: 0.9650108375375834.
[I 2025-11-19 13:01:38,718] Trial 2 finished with value: 0.9650108375375834 and parameters: {'weight_factor': 909.6486217013712, 'learning_rate': 0.016429111579084537, 'max_depth': 12, 'subsample': 0.6793004119536907, 'colsample_bytree': 0.605836

[INFO] Best XGBoost params: {'weight_factor': 261.7292380246308, 'learning_rate': 0.043880622790306106, 'max_depth': 10, 'subsample': 0.9601668590876017, 'colsample_bytree': 0.5002609495133921, 'min_child_weight': 0.11729149355635726, 'gamma': 0.5468482269128541}


In [38]:
evaluate_and_log(model_xgboost_full, X_va_xgboost_full, y_va_xgboost_full, experiment_name=EXPERIMENT_NAME, run_name="XGBoost_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths)

2025/11/19 16:48:02 INFO mlflow.tracking.fluent: Experiment with name 'FRAUD_DETECTION_WITH_CUSTOM_THRESHOLD' does not exist. Creating a new experiment.


[INFO] Logged metrics: {'roc_auc': np.float64(0.9791603126723014), 'pr_auc': np.float64(0.9149830886510049), 'precision': 0.9322847240735804, 'recall': 0.8461166223082507, 'f1': 0.8871131405377981, 'custom_loss': np.float64(0.5406407694652352)}
[INFO] Logged SHAP summary plot.


2025/11/19 16:48:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9791603126723014),
 'pr_auc': np.float64(0.9149830886510049),
 'precision': 0.9322847240735804,
 'recall': 0.8461166223082507,
 'f1': 0.8871131405377981,
 'custom_loss': np.float64(0.5406407694652352)}

In [51]:
evaluate_and_log(model_xgboost_full, X_va_xgboost_full, y_va_xgboost_full, experiment_name=EXPERIMENT_NAME, run_name="XGBoost_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.25)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9791603126723014), 'pr_auc': np.float64(0.9149830886510049), 'precision': 0.8192014356213548, 'recall': 0.8836196467457053, 'f1': 0.8501920614596671, 'custom_loss': np.float64(0.41407863988891525)}
[INFO] Logged SHAP summary plot.


2025/11/20 10:32:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9791603126723014),
 'pr_auc': np.float64(0.9149830886510049),
 'precision': 0.8192014356213548,
 'recall': 0.8836196467457053,
 'f1': 0.8501920614596671,
 'custom_loss': np.float64(0.41407863988891525)}

In [55]:
evaluate_and_log(model_xgboost_full, X_va_xgboost_full, y_va_xgboost_full, experiment_name=EXPERIMENT_NAME, run_name="XGBoost_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.10)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9791603126723014), 'pr_auc': np.float64(0.9149830886510049), 'precision': 0.6127195836044242, 'recall': 0.911444471328333, 'f1': 0.7328080925979963, 'custom_loss': np.float64(0.3300453821925695)}
[INFO] Logged SHAP summary plot.


2025/11/20 10:44:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9791603126723014),
 'pr_auc': np.float64(0.9149830886510049),
 'precision': 0.6127195836044242,
 'recall': 0.911444471328333,
 'f1': 0.7328080925979963,
 'custom_loss': np.float64(0.3300453821925695)}

### 1.2. LightGBM

In [40]:
model_lgbm_full, best_params_lgbm_full, X_va_lgbm_full, y_va_lgbm_full, hist_df, plot_paths = train_lgbm_optuna(train, target)

[I 2025-11-19 16:49:24,757] A new study created in memory with name: lgbm_custom_cost_optimization


[Optuna LightGBM Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-11-19 16:50:38,150] Trial 0 finished with value: 3.4989162462416536 and parameters: {'weight_factor': 954.521448106782, 'num_leaves': 172, 'max_depth': 18, 'learning_rate': 0.04132265854053712, 'feature_fraction': 0.8818189357459641, 'bagging_fraction': 0.6588748156533081, 'bagging_freq': 7, 'min_child_samples': 40, 'lambda_l1': 2.3986236841628236e-06, 'lambda_l2': 5.064261849043662e-07}. Best is trial 0 with value: 3.4989162462416536.
[I 2025-11-19 16:52:23,364] Trial 1 finished with value: 0.618539357890036 and parameters: {'weight_factor': 178.39176647594297, 'num_leaves': 22, 'max_depth': 12, 'learning_rate': 0.23316169143236623, 'feature_fraction': 0.8845352966742596, 'bagging_fraction': 0.842702398015994, 'bagging_freq': 10, 'min_child_samples': 19, 'lambda_l1': 0.0005045902275389655, 'lambda_l2': 1.576931631282666e-05}. Best is trial 1 with value: 0.618539357890036.
[I 2025-11-19 16:54:48,791] Trial 2 finished with value: 0.502512475476488 and parameters: {'weight_factor

[INFO] Best LightGBM params: {'weight_factor': 234.67717851121523, 'num_leaves': 131, 'max_depth': 20, 'learning_rate': 0.16677379165538847, 'feature_fraction': 0.6914915187339126, 'bagging_fraction': 0.9991474900136221, 'bagging_freq': 4, 'min_child_samples': 56, 'lambda_l1': 0.005627970816018173, 'lambda_l2': 6.671477906890939e-08}


In [41]:
evaluate_and_log(model_lgbm_full, X_va_lgbm_full, y_va_lgbm_full, experiment_name=EXPERIMENT_NAME, run_name="LightGBM_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9772242449414609), 'pr_auc': np.float64(0.9182394768279611), 'precision': 0.9612468407750632, 'recall': 0.8282119525768207, 'f1': 0.8897842474655576, 'custom_loss': np.float64(0.6023131371287297)}
[INFO] Logged SHAP summary plot.


2025/11/19 17:38:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9772242449414609),
 'pr_auc': np.float64(0.9182394768279611),
 'precision': 0.9612468407750632,
 'recall': 0.8282119525768207,
 'f1': 0.8897842474655576,
 'custom_loss': np.float64(0.6023131371287297)}

In [52]:
evaluate_and_log(model_lgbm_full, X_va_lgbm_full, y_va_lgbm_full, experiment_name=EXPERIMENT_NAME, run_name="LightGBM_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.25)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9772242449414609), 'pr_auc': np.float64(0.9182394768279611), 'precision': 0.9468313641245972, 'recall': 0.8531333172030002, 'f1': 0.8975435917016673, 'custom_loss': np.float64(0.5156128289362278)}
[INFO] Logged SHAP summary plot.


2025/11/20 10:32:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9772242449414609),
 'pr_auc': np.float64(0.9182394768279611),
 'precision': 0.9468313641245972,
 'recall': 0.8531333172030002,
 'f1': 0.8975435917016673,
 'custom_loss': np.float64(0.5156128289362278)}

In [54]:
evaluate_and_log(model_lgbm_full, X_va_lgbm_full, y_va_lgbm_full, experiment_name=EXPERIMENT_NAME, run_name="LightGBM_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.10)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9772242449414609), 'pr_auc': np.float64(0.9182394768279611), 'precision': 0.9203585147247119, 'recall': 0.8695862569562062, 'f1': 0.8942523015675541, 'custom_loss': np.float64(0.45899515697497206)}
[INFO] Logged SHAP summary plot.


2025/11/20 10:43:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9772242449414609),
 'pr_auc': np.float64(0.9182394768279611),
 'precision': 0.9203585147247119,
 'recall': 0.8695862569562062,
 'f1': 0.8942523015675541,
 'custom_loss': np.float64(0.45899515697497206)}

### 1.3. CatBoost

In [60]:
model_catboost_full, best_params_catboost_full, X_va_catboost_full, y_va_catboost_full, hist_df, plot_paths = train_catboost_optuna(train, target)

KeyboardInterrupt: 

In [44]:
evaluate_and_log(model_catboost_full, X_va_catboost_full, y_va_catboost_full, experiment_name=EXPERIMENT_NAME, run_name="CATboost_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9781869530372198), 'pr_auc': np.float64(0.9081229502984629), 'precision': 0.8664400194269063, 'recall': 0.8632954270505686, 'f1': 0.8648648648648649, 'custom_loss': np.float64(0.4830324787482643)}
[INFO] Logged SHAP summary plot.


2025/11/20 01:47:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9781869530372198),
 'pr_auc': np.float64(0.9081229502984629),
 'precision': 0.8664400194269063,
 'recall': 0.8632954270505686,
 'f1': 0.8648648648648649,
 'custom_loss': np.float64(0.4830324787482643)}

In [ ]:
evaluate_and_log(model_catboost_full, X_va_catboost_full, y_va_catboost_full, experiment_name=EXPERIMENT_NAME, run_name="CATboost_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.25)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9781869530372198), 'pr_auc': np.float64(0.9081229502984629), 'precision': 0.7147534668721109, 'recall': 0.8978949915315751, 'f1': 0.7959249329758713, 'custom_loss': np.float64(0.3698394689606123)}
[INFO] Logged SHAP summary plot.


2025/11/20 10:43:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9781869530372198),
 'pr_auc': np.float64(0.9081229502984629),
 'precision': 0.7147534668721109,
 'recall': 0.8978949915315751,
 'f1': 0.7959249329758713,
 'custom_loss': np.float64(0.3698394689606123)}

In [61]:
evaluate_and_log(model_catboost_full, X_va_catboost_full, y_va_catboost_full, experiment_name=EXPERIMENT_NAME, run_name="CATboost_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.1)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9781869530372198), 'pr_auc': np.float64(0.9081229502984629), 'precision': 0.45759463344513657, 'recall': 0.9242680861359787, 'f1': 0.6121304382661645, 'custom_loss': np.float64(0.30334947675009316)}
[INFO] Logged SHAP summary plot.


2025/11/20 13:35:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9781869530372198),
 'pr_auc': np.float64(0.9081229502984629),
 'precision': 0.45759463344513657,
 'recall': 0.9242680861359787,
 'f1': 0.6121304382661645,
 'custom_loss': np.float64(0.30334947675009316)}

## 2. Reduced Features

In [46]:
X_reduced, feature_ranking = select_important_features(
    train,
    target,
    top_n=100,                # or None to use cumulative 95%
    use_shap=True,            # optional, slower but more accurate
    remove_correlated=True,
    correlation_threshold=0.95
)

[INFO] Training LightGBM for feature importance estimation...
[INFO] Computing SHAP values for refined importance...
[INFO] Keeping top 100 features by importance.
[INFO] Performing correlation pruning (threshold=0.95)...
[INFO] Dropping 15 correlated features.
[DONE] Selected 85 features out of 407.


### 2.1. XGBoost 

In [47]:
model_xgboost_reduced, best_params_xgboost_reduced, X_va_xgboost_reduced, y_va_xgboost_reduced, hist_df, plot_paths = train_xgb_optuna(X_reduced, target, n_trials=30)
evaluate_and_log(model_xgboost_reduced, X_va_xgboost_reduced, y_va_xgboost_reduced, experiment_name=EXPERIMENT_NAME, run_name="XGBoost_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths)
evaluate_and_log(model_xgboost_reduced, X_va_xgboost_reduced, y_va_xgboost_reduced, experiment_name=EXPERIMENT_NAME, run_name="XGBoost_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.25)

[I 2025-11-20 02:07:42,982] A new study created in memory with name: xgboost_customloss_optimization


[Optuna XGBoost Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-11-20 02:10:21,382] Trial 0 finished with value: 0.47448512967185286 and parameters: {'weight_factor': 303.03445169252836, 'learning_rate': 0.10083183895081857, 'max_depth': 11, 'subsample': 0.8963608060733238, 'colsample_bytree': 0.6785103891501436, 'min_child_weight': 0.07640426499273617, 'gamma': 4.238768344859967}. Best is trial 0 with value: 0.47448512967185286.
[I 2025-11-20 02:15:17,159] Trial 1 finished with value: 0.47430942625086236 and parameters: {'weight_factor': 384.04306259406184, 'learning_rate': 0.043534070489637824, 'max_depth': 14, 'subsample': 0.7174468047365997, 'colsample_bytree': 0.7170701546516388, 'min_child_weight': 0.45039260296668343, 'gamma': 0.9937966768189904}. Best is trial 1 with value: 0.47430942625086236.
[I 2025-11-20 02:21:49,082] Trial 2 finished with value: 0.438930817423392 and parameters: {'weight_factor': 384.6739597646329, 'learning_rate': 0.029851398336222162, 'max_depth': 11, 'subsample': 0.9780693528395272, 'colsample_bytree': 0.665

[INFO] Best XGBoost params: {'weight_factor': 563.1612148501741, 'learning_rate': 0.016063616494696824, 'max_depth': 15, 'subsample': 0.9610875639100243, 'colsample_bytree': 0.571910755843421, 'min_child_weight': 0.0054253161383116325, 'gamma': 3.032335195843312}
[INFO] Logged metrics: {'roc_auc': np.float64(0.9783492682732146), 'pr_auc': np.float64(0.8923424129990234), 'precision': 0.8718726307808946, 'recall': 0.8347447374788289, 'f1': 0.8529048207663782, 'custom_loss': np.float64(0.5825769634571748)}
[INFO] Logged SHAP summary plot.


2025/11/20 05:15:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.
[INFO] Logged metrics: {'roc_auc': np.float64(0.9783492682732146), 'pr_auc': np.float64(0.8923424129990234), 'precision': 0.7010428736964078, 'recall': 0.8782966368255505, 'f1': 0.7797229083879282, 'custom_loss': np.float64(0.5825769634571748)}
[INFO] Logged SHAP summary plot.


2025/11/20 05:16:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9783492682732146),
 'pr_auc': np.float64(0.8923424129990234),
 'precision': 0.7010428736964078,
 'recall': 0.8782966368255505,
 'f1': 0.7797229083879282,
 'custom_loss': np.float64(0.5825769634571748)}

### 2.2. LightGBM

In [48]:
model_lgmb_reduced, best_params_lgmb_reduced, X_va_lgmb_reduced, y_va_lgmb_reduced, hist_df, plot_paths = train_lgbm_optuna(X_reduced, target, n_trials=30)
evaluate_and_log(model_lgmb_reduced, X_va_lgmb_reduced, y_va_lgmb_reduced, experiment_name=EXPERIMENT_NAME, run_name="LGBM_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths)
evaluate_and_log(model_lgmb_reduced, X_va_lgmb_reduced, y_va_lgmb_reduced, experiment_name=EXPERIMENT_NAME, run_name="LGBM_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.25)

[I 2025-11-20 05:16:45,506] A new study created in memory with name: lgbm_custom_cost_optimization


[Optuna LightGBM Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-11-20 05:17:31,240] Trial 0 finished with value: 0.6125331219595925 and parameters: {'weight_factor': 988.4441409271524, 'num_leaves': 166, 'max_depth': 20, 'learning_rate': 0.18528910883777497, 'feature_fraction': 0.9538139183876564, 'bagging_fraction': 0.7829489483618187, 'bagging_freq': 5, 'min_child_samples': 65, 'lambda_l1': 3.14241707785604e-05, 'lambda_l2': 0.011144987897729686}. Best is trial 0 with value: 0.6125331219595925.
[I 2025-11-20 05:17:50,230] Trial 1 finished with value: 3.4989162462416536 and parameters: {'weight_factor': 763.9845955234861, 'num_leaves': 115, 'max_depth': 19, 'learning_rate': 0.03601466570668402, 'feature_fraction': 0.9765467245612626, 'bagging_fraction': 0.6274827760378981, 'bagging_freq': 8, 'min_child_samples': 58, 'lambda_l1': 1.4364084634718396e-07, 'lambda_l2': 0.10126313290615058}. Best is trial 0 with value: 0.6125331219595925.
[I 2025-11-20 05:18:13,468] Trial 2 finished with value: 0.8158042208441103 and parameters: {'weight_factor

[INFO] Best LightGBM params: {'weight_factor': 595.0221512419944, 'num_leaves': 76, 'max_depth': 20, 'learning_rate': 0.19236941161792298, 'feature_fraction': 0.879964550074313, 'bagging_fraction': 0.9215867301351228, 'bagging_freq': 4, 'min_child_samples': 84, 'lambda_l1': 8.965510102130184, 'lambda_l2': 1.257085921083824e-06}
[INFO] Logged metrics: {'roc_auc': np.float64(0.9737186890783829), 'pr_auc': np.float64(0.8897176420223434), 'precision': 0.8477306002928258, 'recall': 0.8405516573917251, 'f1': 0.8441258656299356, 'custom_loss': np.float64(0.5632471974802723)}
[INFO] Logged SHAP summary plot.


2025/11/20 05:31:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.
[INFO] Logged metrics: {'roc_auc': np.float64(0.9737186890783829), 'pr_auc': np.float64(0.8897176420223434), 'precision': 0.7289436477479297, 'recall': 0.8732155819017663, 'f1': 0.7945838837516512, 'custom_loss': np.float64(0.5632471974802723)}
[INFO] Logged SHAP summary plot.


2025/11/20 05:32:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9737186890783829),
 'pr_auc': np.float64(0.8897176420223434),
 'precision': 0.7289436477479297,
 'recall': 0.8732155819017663,
 'f1': 0.7945838837516512,
 'custom_loss': np.float64(0.5632471974802723)}

### 2.3. CatBoost

In [49]:
model_catoost_reduced, best_params_catoost_reduced, X_va_catoost_reduced, y_va_catoost_reduced, hist_df, plot_paths = train_catboost_optuna(X_reduced, target, n_trials=30)
evaluate_and_log(model_catoost_reduced, X_va_catoost_reduced, y_va_lgmb_reduced, experiment_name=EXPERIMENT_NAME, run_name="CATBoost_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths)
evaluate_and_log(model_catoost_reduced, X_va_catoost_reduced, y_va_lgmb_reduced, experiment_name=EXPERIMENT_NAME, run_name="CATBoost_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.25)

[I 2025-11-20 05:32:09,180] A new study created in memory with name: catboost_custom_cost_optimization


[Optuna CatBoost Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-11-20 05:35:30,115] Trial 0 finished with value: 0.9650108375375834 and parameters: {'weight_factor': 163.7397266034792, 'learning_rate': 0.03120659802284773, 'depth': 12, 'l2_leaf_reg': 8.181175589517238, 'subsample': 0.6736681243794981, 'border_count': 107}. Best is trial 0 with value: 0.9650108375375834.
[I 2025-11-20 05:42:17,422] Trial 1 finished with value: 0.9649769704447559 and parameters: {'weight_factor': 974.137254461418, 'learning_rate': 0.24457363544379654, 'depth': 16, 'l2_leaf_reg': 4.954780337206101, 'subsample': 0.874814049584014, 'border_count': 54}. Best is trial 1 with value: 0.9649769704447559.
[I 2025-11-20 05:49:22,039] Trial 2 finished with value: 0.9649515700355265 and parameters: {'weight_factor': 829.7542285570263, 'learning_rate': 0.1173314516038542, 'depth': 15, 'l2_leaf_reg': 5.661806434494909, 'subsample': 0.9543732606288229, 'border_count': 97}. Best is trial 2 with value: 0.9649515700355265.
[I 2025-11-20 05:55:44,572] Trial 3 finished with valu

[INFO] Best CatBoost params: {'weight_factor': 826.5125110443701, 'learning_rate': 0.05013487336385276, 'depth': 10, 'l2_leaf_reg': 2.1356969501070076, 'subsample': 0.7829974303923855, 'border_count': 139}
[INFO] Logged metrics: {'roc_auc': np.float64(0.9726674973133655), 'pr_auc': np.float64(0.8668056153521687), 'precision': 0.6505289582212659, 'recall': 0.8778127268328091, 'f1': 0.7472708547888774, 'custom_loss': np.float64(0.44407660785044195)}
[INFO] Logged SHAP summary plot.


2025/11/20 07:45:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.
[INFO] Logged metrics: {'roc_auc': np.float64(0.9726674973133655), 'pr_auc': np.float64(0.8668056153521687), 'precision': 0.3822463768115942, 'recall': 0.9189450762158239, 'f1': 0.539910441395977, 'custom_loss': np.float64(0.44407660785044195)}
[INFO] Logged SHAP summary plot.


2025/11/20 07:48:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9726674973133655),
 'pr_auc': np.float64(0.8668056153521687),
 'precision': 0.3822463768115942,
 'recall': 0.9189450762158239,
 'f1': 0.539910441395977,
 'custom_loss': np.float64(0.44407660785044195)}

## 3. Ensemble

In [ ]:
base_models, X_tr, X_va, y_tr, y_va, X_test, y_test, best_params_dict = train_best_base_models_from_mlflow(
    train, target, test_size=0.15 ,experiment_name=EXPERIMENT_NAME
)

[INFO] Searching for best xgboost model in experiment 'FRAUD_DETECTION'...
[INFO] Found best xgboost run 2de7afde847a4e978f4d7914a09c9200 with pr_auc=0.9136
[INFO] Retraining xgboost with best parameters...
[INFO] Searching for best catboost model in experiment 'FRAUD_DETECTION'...
[INFO] Found best catboost run 3c8041e4dcf64b9c84c2d5bdd039c60c with pr_auc=0.9082
[INFO] Retraining catboost with best parameters...
[INFO] Searching for best lightgbm model in experiment 'FRAUD_DETECTION'...
[INFO] Found best lightgbm run cd33e5f9d37a4660a9621d0bed29f886 with pr_auc=0.9174
[INFO] Retraining lightgbm with best parameters...


### 3.1. Logistic

In [ ]:
ensemble, base_models, X_meta_va, y_va, X_meta_test, y_test, best_params_log, hist_df, plot_paths = train_ensemble(
    base_models, X_tr, y_tr, X_va, y_va, X_test, y_test, n_trials=50
)

[INFO] Generating meta-features...
[INFO] xgboost validation PR-AUC: 0.8901
[INFO] catboost validation PR-AUC: 0.8128


[I 2025-11-18 03:06:15,837] A new study created in memory with name: ensemble_meta_learner_pr_auc_optimization


[INFO] lightgbm validation PR-AUC: 0.8840
[INFO] Optimizing meta-learner with Optuna...


[Optuna Ensemble Meta-Learner Tuning]:   0%|          | 0/50 [00:00<?, ?trial/s]

[I 2025-11-18 03:06:17,544] Trial 0 finished with value: 0.8968493565501713 and parameters: {'C': 0.004650653818750843, 'class_weight': None, 'solver': 'saga', 'penalty_saga': 'elasticnet', 'l1_ratio': 0.9262925490300581}. Best is trial 0 with value: 0.8968493565501713.
[I 2025-11-18 03:06:18,422] Trial 1 finished with value: 0.8967225113225421 and parameters: {'C': 0.07273320235255908, 'class_weight': 'balanced', 'solver': 'newton-cg', 'penalty_newton-cg': 'l2'}. Best is trial 0 with value: 0.8968493565501713.
[I 2025-11-18 03:06:20,092] Trial 2 finished with value: 0.8969843620283047 and parameters: {'C': 0.05786152184681003, 'class_weight': None, 'solver': 'saga', 'penalty_saga': 'l2'}. Best is trial 2 with value: 0.8969843620283047.
[I 2025-11-18 03:08:18,967] Trial 3 finished with value: 0.7973015325222292 and parameters: {'C': 29.13605449340287, 'class_weight': 'balanced', 'solver': 'sag', 'penalty_sag': None}. Best is trial 2 with value: 0.8969843620283047.
[I 2025-11-18 03:08:1

[INFO] Best meta-learner params: {'C': 0.21957921937861558, 'class_weight': None, 'solver': 'lbfgs', 'penalty_lbfgs': None}
[INFO] Ensemble validation PR-AUC: 0.8972
[INFO] Ensemble validation ROC-AUC: 0.9778
[INFO] Ensemble test PR-AUC: 0.8934
[INFO] Ensemble test ROC-AUC: 0.9770


In [54]:
evaluate_and_log(ensemble, pd.DataFrame(X_meta_va), y_va, experiment_name=EXPERIMENT_NAME, run_name="Ensemble_from_best_base_models", hp_search_history=hist_df)

[INFO] Logged metrics: {'roc_auc': np.float64(0.977775749485985), 'pr_auc': np.float64(0.8971915104190178), 'precision': 0.9516616314199395, 'recall': 0.7621582385676264, 'f1': 0.8464328899637243, 'custom_loss': np.float64(0.8336437836556372)}
[WARN] SHAP skipped: Model type not yet supported by TreeExplainer: <class 'sklearn.linear_model._logistic.LogisticRegression'>


2025/11/18 03:11:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.977775749485985),
 'pr_auc': np.float64(0.8971915104190178),
 'precision': 0.9516616314199395,
 'recall': 0.7621582385676264,
 'f1': 0.8464328899637243,
 'custom_loss': np.float64(0.8336437836556372)}

### 3.2. NN

In [55]:
nn_model, base_mdls, X_val, y_val, X_test, y_test, params, hist, plots = train_neural_network_ensemble(
    base_models=base_models,
    X_tr=X_tr,
    y_tr=y_tr,
    X_va=X_va,
    y_va=y_va,
    X_test=X_test,
    y_test=y_test,
    n_trials=30,
    epochs=100
)

[INFO] Generating meta-features from base models...
[INFO] xgboost validation custom_loss: 0.8598
[INFO] catboost validation custom_loss: 1.3679


[I 2025-11-18 03:12:01,112] A new study created in memory with name: neural_network_ensemble_optimization


[INFO] lightgbm validation custom_loss: 0.9562
[INFO] Optimizing neural network meta-learner with Optuna...


[Optuna NN Ensemble Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-11-18 03:16:57,637] Trial 0 finished with value: 0.7586107630304467 and parameters: {'n_layers': 1, 'hidden_size_0': 180, 'dropout_rate': 0.38574414781037836, 'learning_rate': 0.005570027698867698, 'weight_decay': 0.000253232244112943}. Best is trial 0 with value: 0.7586107630304467.
[I 2025-11-18 03:26:37,819] Trial 1 finished with value: 0.7185118704914146 and parameters: {'n_layers': 1, 'hidden_size_0': 38, 'dropout_rate': 0.2716836549595695, 'learning_rate': 0.0007150539132222626, 'weight_decay': 5.689583047950391e-06}. Best is trial 1 with value: 0.7185118704914146.
[I 2025-11-18 03:37:02,296] Trial 2 finished with value: 0.6539946489653538 and parameters: {'n_layers': 2, 'hidden_size_0': 200, 'hidden_size_1': 138, 'dropout_rate': 0.014344713117248387, 'learning_rate': 0.009775444648925805, 'weight_decay': 4.869795488201027e-05}. Best is trial 2 with value: 0.6539946489653538.
[I 2025-11-18 03:42:49,435] Trial 3 finished with value: 0.8218664273376909 and parameters: {'n_l

[INFO] Best neural network params: {'n_layers': 3, 'hidden_size_0': 64, 'hidden_size_1': 149, 'hidden_size_2': 123, 'dropout_rate': 0.1519020286431952, 'learning_rate': 0.006663695072321037, 'weight_decay': 1.8779452673203788e-06}
[INFO] Neural network ensemble validation custom_loss: 0.6392
[INFO] Neural network ensemble validation PR-AUC: 0.8229
[INFO] Neural network ensemble validation ROC-AUC: 0.9408
[INFO] Neural network ensemble test custom_loss: 0.6862
[INFO] Neural network ensemble test PR-AUC: 0.8160
[INFO] Neural network ensemble test ROC-AUC: 0.9344
